In [63]:
def load_data(filename):
    """ returns list of tokens from filename """
    data = []
    with open(filename, "r") as file:
        for line in file:
            for token in line.split(' '):
                token = token.rstrip('\n')
                if token != '':
                    data.append(token)
    return data

def get_grams(data):
    """ Gets unigrams and bigrams """
    bigrams = {}
    unigrams = {}
    for i in range(len(data)-1):        
        if (data[i], data[i+1]) in bigrams:
            bigrams[(data[i], data[i + 1])] += 1
        else:
            bigrams[(data[i], data[i + 1])] = 1
        if data[i] in unigrams:
            unigrams[data[i]] += 1
        else:
            unigrams[data[i]] = 1
    return unigrams, bigrams

In [75]:
def get_bi_probs(unigrams, bigrams):
    """ Uses entire vocab and bigrams to get the probablily of each bigram"""
    bigram_probs = {}
    for bigram in bigrams.keys():
        word1 = bigram[0]
        word2 = bigram[1]
        bigram_probs[bigram] = (bigrams.get(bigram))/(unigrams.get(word1))
    return bigram_probs

In [81]:
train_data = load_data('data/hw2_training_sets.txt')
unigrams, bigrams = get_grams(train_data)
bigram_probs = get_bi_probs(unigrams, bigrams)

In [82]:
test_data = load_data("data/test_set.txt")
get_grams(test_data)

({'When': 2,
  'trouble': 1,
  'or': 1,
  'persecution': 1,
  'comes': 1,
  'because': 1,
  'of': 4,
  'the': 11,
  'word': 1,
  ',': 9,
  'they': 4,
  'quickly': 1,
  'fall': 1,
  'away': 1,
  '.': 4,
  'The': 1,
  'people': 1,
  'were': 2,
  'amazed': 1,
  'when': 1,
  'saw': 2,
  'mute': 1,
  'speaking': 1,
  'crippled': 1,
  'made': 1,
  'well': 1,
  'lame': 1,
  'walking': 1,
  'and': 3,
  'blind': 1,
  'seeing': 1,
  'Pilate': 1,
  'that': 2,
  'he': 2,
  'was': 3,
  'getting': 1,
  'nowhere': 1,
  'but': 1,
  'instead': 1,
  'an': 1,
  'uproar': 1,
  'starting': 1,
  'took': 1,
  'water': 1,
  'washed': 1,
  'his': 2,
  'hands': 1,
  'in': 2,
  'front': 1,
  'crowd': 1,
  'As': 1,
  'coming': 1,
  'down': 1,
  'mountain': 1,
  'lord': 1,
  'gave': 1,
  'them': 1,
  'orders': 1,
  'not': 1,
  'to': 1,
  'tell': 1,
  'anyone': 1,
  'what': 1,
  'had': 2,
  'seen': 1,
  'until': 1,
  'son': 1,
  'Man': 1,
  'risen': 1,
  'from': 3,
  'dead': 1,
  'A': 1,
  'certain': 1,
  'man': 1,

In [9]:
# Let's put it all together. 

# 1) Load the data, and shuffle the training data.
# TODO
vocab, train_data = load_data('data/hw2_training_sets.txt')
print(len(vocab))
# _, dev_data = load_data('data/bible.dev.txt', vocab)
# print(len(vocab))
_, test_data = load_data('data/test_set.txt', vocab)
print(len(vocab))

print(train_data[0])
random.shuffle(train_data)
print(train_data[0])

7146
7146
(1, 2)
(1203, 115)


In [25]:
# 2) Initialize our model.
# TODO

our_lm = FeedforwardLM(len(vocab), 10, 15)
count_parameters(our_lm)
#print_parameters(our_lm)

183465

In [29]:
# Note: Perplexity is just exp(2, cross-entropy). So we just use the loss here.
def validate(model, data):
    
    model.eval()
    
    # TODO: Implement validation function
    av_loss = 0
    for (x, y) in data[:1000]:
        
        # a) calculate probs / get an output
        y_raw = model(x)
        
        # b) compute loss
        loss = ce(y_raw.unsqueeze(0),y.unsqueeze(0))
        av_loss += loss

    av_loss = av_loss/len(data[:1000])
    
    print("Average loss: " + str(av_loss))
    

In [30]:
# 3) Now we train our model. 
# TODO

epochs = 10
ce = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=0)
optimizer = optim.SGD(our_lm.parameters(), lr=0.1)

for i in range(epochs):
    print('### Epoch: ' + str(i+1) + ' ###')
    av_loss = 0
    our_lm.train()
    for (x, y) in train_data[:10000]:
        optimizer.zero_grad()
        
        # a) calculate probs / get an output
        y_raw = our_lm(x)
        y_hat = softmax(y_raw)
        
        # b) compute loss
        loss = ce(y_raw.unsqueeze(0),y.unsqueeze(0))
        av_loss += loss
        
        # c) get the gradient
        loss.backward()

        # d) update the weights
        optimizer.step()
    validate(our_lm, dev_data)
    print(av_loss/len(train_data[:10000]))

### Epoch: 1 ###
Average loss: tensor(6.4516, grad_fn=<DivBackward0>)
tensor(5.3513, grad_fn=<DivBackward0>)
### Epoch: 2 ###
Average loss: tensor(6.5446, grad_fn=<DivBackward0>)
tensor(5.1827, grad_fn=<DivBackward0>)
### Epoch: 3 ###
Average loss: tensor(6.6212, grad_fn=<DivBackward0>)
tensor(5.0426, grad_fn=<DivBackward0>)
### Epoch: 4 ###
Average loss: tensor(6.7176, grad_fn=<DivBackward0>)
tensor(4.9312, grad_fn=<DivBackward0>)
### Epoch: 5 ###
Average loss: tensor(6.8294, grad_fn=<DivBackward0>)
tensor(4.8364, grad_fn=<DivBackward0>)
### Epoch: 6 ###
Average loss: tensor(6.9647, grad_fn=<DivBackward0>)
tensor(4.7584, grad_fn=<DivBackward0>)
### Epoch: 7 ###
Average loss: tensor(7.0675, grad_fn=<DivBackward0>)
tensor(4.6981, grad_fn=<DivBackward0>)
### Epoch: 8 ###
Average loss: tensor(7.2170, grad_fn=<DivBackward0>)
tensor(4.6456, grad_fn=<DivBackward0>)
### Epoch: 9 ###
Average loss: tensor(7.3021, grad_fn=<DivBackward0>)
tensor(4.6144, grad_fn=<DivBackward0>)
### Epoch: 10 ###
A

In [ ]:
# Use the model to predict some words!
# TODO

words = ['walk', 'to']

for i in range(10):
    word = words[i]
    idx = vocab[word]
    tensor_idx = torch.tensor(idx)
    
    raw_output = our_lm(tensor_idx)
    probs = softmax(raw_output)
    
    pred = torch.argmax(probs)
    
    # Print prediction.
    for w, v in vocab.items():
        if v == pred:
            print(word + ' ' + w)
            words.append(w)
    

In [ ]:
# Note: Perplexity is just exp(2, cross-entropy). So we just use the loss here.
def validate(model, data):
    
    model.eval()
    
    # TODO: Implement validation function
    av_loss = 0
    for (x, y) in data[:1000]:
        
        # a) calculate probs / get an output
        y_raw = model(x)
        
        # b) compute loss
        loss = ce(y_raw.unsqueeze(0),y.unsqueeze(0))
        av_loss += loss

    av_loss = av_loss/len(data[:1000])
    
    print("Average loss: " + str(av_loss))
    